In [ ]:
import numpy as np
import random

from models.Coordinate import Coordinate
import config


In [ ]:
def read_data(file_path):
    list_cities = [None]

    with open(file_path, 'r') as f:
        data = f.readlines()
        NUMBER_CITIES = int(data[3].split(':')[1].strip())

        for line in data[6: 6 + NUMBER_CITIES]:
            _, x, y = list(map(int, line.split()))
            list_cities.append(Coordinate(x, y))

    return list_cities, NUMBER_CITIES


list_cities, NUMBER_CITIES = read_data(config.DATA_PATH)
print(list_cities, NUMBER_CITIES)


In [ ]:
def init_population():
    population = [None]

    for _ in range(1, config.POPULATION_SIZE + 1):
        while True:
            individual = random.sample(
                range(1, NUMBER_CITIES + 1), NUMBER_CITIES)
            if individual not in population:
                break
        population.append(individual)

    return population


population = init_population()
print(population)


In [ ]:
DISTANCE_MATRIX = [[None] * (NUMBER_CITIES + 1)] * (NUMBER_CITIES + 1)


def euclidean_distance(index_city1, index_city2):
    if not DISTANCE_MATRIX[index_city1][index_city2]:
        DISTANCE_MATRIX[index_city1][index_city2] = np.linalg.norm((list_cities[index_city1].x - list_cities[index_city2].x,
                                                                    list_cities[index_city1].y - list_cities[index_city2].y))

    return DISTANCE_MATRIX[index_city1][index_city2]


def cost(individual):
    if not individual:
        return 0
    c = 0

    for i in range(len(individual) - 1):
        c += euclidean_distance(individual[i], individual[i+1])
    # Last city with first city
    c += euclidean_distance(individual[-1], individual[0])

    return c


def find_best(population):
    best_individual = population[1]
    best_cost = cost(population[1])
    for individual in population[2:]:
        tmp = cost(individual)
        if tmp < best_cost:
            best_individual = individual
            best_cost = tmp
    return best_individual, best_cost


In [ ]:
def select_parents(population):
    candidates = random.sample(population[1:], 4)
    return (candidates[0] if cost(candidates[0]) < cost(candidates[1]) else candidates[1],
            candidates[2] if cost(candidates[2]) < cost(candidates[3]) else candidates[3])


def crossover(parents):
    if random.random() < config.P_CROSSOVER:
        parent1, parent2 = parents[0], parents[1]
        children1, children2 = [None] * len(parent1), [None] * len(parent1)
        for i in range(len(parent1)):
            cycle1 = []
            while parent2[i] not in cycle1:
                if children1[i]:
                    continue
                cycle1.append(parent1[i])
                i = parent2.index(parent1[i])
        
        # return children1
        parent1, parent2 = parents[0].copy(), parents[1].copy()
        list_mem = []
        for i in range(len(parent1)):
            cycle = []
            while parent2[i] not in cycle:
                cycle.append(parent1[i])
                i = parent2.index(parent1[i])
    return None, None

def mutation(parents):
    if random.random() < config.P_MUTATION:
        result = []
        for parent in parents:
            pos1, pos2 = tuple(random.sample(range(len(parent)), 2))
            parent[pos1], parent[pos2] = parent[pos2], parent[pos1]
            result.append(parent)
        return tuple(result)
    return None, None


def gen_offspring(population):
    offspring = [None]
    
    while len(offspring) < config.POPULATION_SIZE + 1:
        parents = select_parents(population)

        children = crossover(parents)
        if children[0]:
            for child in children:
                offspring.append(child)
            # Mutation if only crossover
            children = mutation(parents)
            if children[0]:
                for child in children:
                    offspring.append(child)

    random_index_remove = random.randint(0, len(offspring) - 1)
    offspring[random_index_remove] = find_best(population)[0]

    return offspring


In [ ]:
history = [None]

for i in range(config.NUM_GENERATIONS):
    history.append(find_best(population))
    offspring = gen_offspring(population)
    population = offspring


In [ ]:
print(history[-1])